In [ ]:
pip install pyqt5


In [2]:
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QPushButton, QVBoxLayout, QHBoxLayout, QInputDialog, 
                             QGraphicsScene, QGraphicsView, QGraphicsEllipseItem, QGraphicsLineItem, QGraphicsTextItem)
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QPen, QColor, QPainter
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas

# Node Class
class Node(QGraphicsEllipseItem):
    def __init__(self, x, y, node_id):
        super().__init__(-20, -20, 40, 40)  # Node size
        self.setPos(x, y)
        self.setBrush(QColor(3, 169, 244))  # Blue color for nodes
        self.setFlags(QGraphicsEllipseItem.ItemIsMovable | QGraphicsEllipseItem.ItemIsSelectable)
        self.node_id = node_id
        self.text = QGraphicsTextItem(node_id, self)
        self.text.setDefaultTextColor(Qt.black)
        self.text.setPos(-10, -10)  # Adjust text position inside the node

    def mouseReleaseEvent(self, event):
        super().mouseReleaseEvent(event)
        # Ensure edges connected to this node are updated
        for edge in self.scene().items():
            if isinstance(edge, Edge):
                edge.update_position()

# Edge Class
class Edge(QGraphicsLineItem):
    def __init__(self, start_node, end_node, cash_flow):
        super().__init__()
        self.start_node = start_node
        self.end_node = end_node
        self.setPen(QPen(Qt.gray, 2))
        self.cash_flow = cash_flow
        self.text = QGraphicsTextItem(f"${cash_flow}", self)
        self.text.setDefaultTextColor(Qt.green)
        self.update_position()

    def update_position(self):
        # Ensure the edge is drawn between the start and end nodes
        start_pos = self.start_node.scenePos()
        end_pos = self.end_node.scenePos()
        self.setLine(start_pos.x() + 20, start_pos.y() + 20, end_pos.x() + 20, end_pos.y() + 20)
        self.update_text_position()

    def update_text_position(self):
        # Display the cash flow text in the middle of the edge
        mid_x = (self.start_node.scenePos().x() + self.end_node.scenePos().x()) / 2
        mid_y = (self.start_node.scenePos().y() + self.end_node.scenePos().y()) / 2
        self.text.setPos(mid_x, mid_y)

# Main App
class CashFlowApp(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Cash Flow Minimizer")
        self.setGeometry(100, 100, 1200, 600)

        main_layout = QHBoxLayout(self)

        # Left half: Node and edge inputs
        self.input_scene = QGraphicsScene()
        self.input_view = QGraphicsView(self.input_scene)
        self.input_view.setRenderHint(QPainter.Antialiasing)
        self.input_view.setSceneRect(0, 0, 600, 600)

        input_control_layout = QVBoxLayout()
        self.add_node_btn = QPushButton("Add Node")
        self.add_node_btn.clicked.connect(self.add_node)
        input_control_layout.addWidget(self.add_node_btn)

        self.add_edge_btn = QPushButton("Add Edge")
        self.add_edge_btn.clicked.connect(self.add_edge)
        input_control_layout.addWidget(self.add_edge_btn)

        self.clear_btn = QPushButton("Clear All")
        self.clear_btn.clicked.connect(self.clear_all)
        input_control_layout.addWidget(self.clear_btn)

        left_layout = QVBoxLayout()
        left_layout.addWidget(self.input_view)
        left_layout.addLayout(input_control_layout)

        # Right half: Processed graph output
        self.output_canvas = FigureCanvas(plt.Figure())
        right_layout = QVBoxLayout()
        right_layout.addWidget(self.output_canvas)

        self.process_btn = QPushButton("Process Graph")
        self.process_btn.clicked.connect(self.process_graph)
        right_layout.addWidget(self.process_btn)

        main_layout.addLayout(left_layout)
        main_layout.addLayout(right_layout)

        self.setLayout(main_layout)
        self.nodes = []
        self.edges = []
        self.graph = nx.DiGraph()  # Directed graph to represent cash flows

    def add_node(self):
        # Add new node with an incremented ID
        node_id = f"Node {len(self.nodes) + 1}"
        node = Node(300, 300, node_id)  # Default position for new nodes
        self.input_scene.addItem(node)
        self.nodes.append(node)
        self.graph.add_node(node_id)

    def add_edge(self):
        # Add an edge between two selected nodes with cash flow
        node_ids = [node.node_id for node in self.nodes]
        start_node_id, ok1 = QInputDialog.getItem(self, "Select Start Node", "Start Node:", node_ids, 0, False)
        if not ok1:
            return
        end_node_id, ok2 = QInputDialog.getItem(self, "Select End Node", "End Node:", node_ids, 0, False)
        if not ok2:
            return
        cash_flow, ok3 = QInputDialog.getText(self, "Cash Flow", "Enter cash flow:")
        if not ok3 or not cash_flow.isdigit():
            return

        start_node = next(node for node in self.nodes if node.node_id == start_node_id)
        end_node = next(node for node in self.nodes if node.node_id == end_node_id)
        edge = Edge(start_node, end_node, int(cash_flow))  # Ensure cash flow is an integer
        self.input_scene.addItem(edge)
        self.edges.append(edge)
        self.input_scene.addItem(edge.text)

        self.graph.add_edge(start_node_id, end_node_id, weight=int(cash_flow))

    def clear_all(self):
        # Clear the scene and reset everything
        self.input_scene.clear()
        self.nodes.clear()
        self.edges.clear()
        self.graph.clear()

    def process_graph(self):
        # Ensure there are nodes to process
        if not self.nodes:
            print("No nodes available for processing")
            return
        print("Processing...")

        minimized_edges = self.minimize_cash_flows()

        # Check if we have any minimized edges to display
        if minimized_edges:
            self.show_minimized_graph(minimized_edges)
        else:
            print("No minimized edges to show")

    def minimize_cash_flows(self):
        # Greedy algorithm for minimizing cash flows
        net_balance = {node: 0 for node in self.graph.nodes()}

        # Calculate net balance for each node
        for u, v, data in self.graph.edges(data=True):
            net_balance[u] -= data['weight']  # Outgoing cash flow
            net_balance[v] += data['weight']  # Incoming cash flow

        # Separate into creditors (positive balance) and debtors (negative balance)
        creditors = [(node, balance) for node, balance in net_balance.items() if balance > 0]
        debtors = [(node, balance) for node, balance in net_balance.items() if balance < 0]

        # Sort creditors and debtors
        creditors.sort(key=lambda x: -x[1])  # Sort by descending balance
        debtors.sort(key=lambda x: x[1])  # Sort by ascending (more negative) balance

        minimized_graph = nx.DiGraph()

        # Match debtors and creditors greedily
        i, j = 0, 0
        while i < len(debtors) and j < len(creditors):
            debtor, debt = debtors[i]
            creditor, credit = creditors[j]
            settle_amount = min(-debt, credit)

            # Add the settled amount as an edge from debtor to creditor
            minimized_graph.add_edge(debtor, creditor, weight=settle_amount)

            # Update balances
            debtors[i] = (debtor, debt + settle_amount)
            creditors[j] = (creditor, credit - settle_amount)

            if debtors[i][1] == 0:
                i += 1
            if creditors[j][1] == 0:
                j += 1

        return minimized_graph.edges(data=True)

    def show_minimized_graph(self, minimized_edges):
        # Clear the existing plot
        self.output_canvas.figure.clear()
        ax = self.output_canvas.figure.add_subplot(111)
        ax.set_title("Minimized Cash Flows")

        # Create a new minimized graph
        minimized_graph = nx.DiGraph()

        for u, v, data in minimized_edges:
            minimized_graph.add_edge(u, v, weight=data['weight'])

        pos = nx.spring_layout(minimized_graph)  # Layout for the minimized graph

        # Draw nodes and edges for the minimized graph
        nx.draw(minimized_graph, pos, ax=ax, with_labels=True, node_size=2000, node_color="lightgreen")
        
        # Create edge labels with minimized cash flows
        labels = {(u, v): f"${data['weight']}" for u, v, data in minimized_edges}
        nx.draw_networkx_edge_labels(minimized_graph, pos, ax=ax, edge_labels=labels)

        self.output_canvas.draw()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = CashFlowApp()
    window.show()
    sys.exit(app.exec_())


Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...


SystemExit: 0